In [11]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [ ]:
def cnn_model_fn(features, labels, mode):
    
    input_layer = tf.reshape(features["x"], [-1, 64, 64, 1])
    
    # First convolutional layer
    conv1 = tf.layers.conv2d(
            inputs=input_layer,
            filters=32,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu)
    
    # First Pooling layer
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2)
    
    # Second convolutional layer
    conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=64,
            kernel_size=[5, 5],
            padding="same",
            activation=tf.nn.relu)
    
    # Second Pooling layer
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2)
    
    # Flat layer
    pool2_flat = tf.reshape(pool2, [-1, 16 * 16 * 64])
    
    # Dense layer
    dense1 = tf.layers.dense(inputs=pool2_flat, units=256, activation=tf.nn.relu)
    
    
    # dropping out operation
#     dropout = tf.layers.dropout(
#         inputs=dense1, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    # logits layer
    logits = tf.layers.dense(inputs=dropout, units=2)
    
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=logits)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.mean_squared_error(labels=labels,predictions=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=logits)}
    return tf.estimator.EstimatorSpec(
        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [13]:
def preprocess(x, y):
    x = x.reshape(x.shape[0], 64, 64, 1)
    share = 400
    x_train = x[:share]
    y_train = y[:share]
    x_val = x[share:]
    y_val = y[share:]
    return x_train, y_train, x_val, y_val


In [ ]:
def main(unused_argv):
  # Load training and eval data
  image = np.load("clock_image.npy") 
  label = np.load("clock_time.npy")
  image  = tf.keras.utils.normalize(image, axis=1)  # scales data between 0 and 1

  train_data, train_labels, eval_data, eval_labels = preprocess(image, label)

  # Create the Estimator
  classifier = tf.estimator.Estimator(
      model_fn=cnn_model_fn, model_dir=None)


  # Train the model
  train_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
      x={"x": train_data},
      y=train_labels,
      batch_size=16,
      num_epochs=60,
      shuffle=True)

  classifier.train(
      input_fn=train_input_fn,
      steps=20000)

  # Evaluate the model and print results
  eval_input_fn = tf.compat.v1.estimator.inputs.numpy_input_fn(
      x={"x": eval_data}, y=eval_labels, num_epochs=1, shuffle=False)
  eval_results = classifier.evaluate(input_fn=eval_input_fn)
  print(eval_results)

if __name__ == "__main__":
  tf.app.run()
  # 不能用cross entropy
  # 2 output layer
  # 做成regression -》 不能用cross entropy，用MSE
  # 更了解linear和logistic regression

  # 不要probability，改loss